In [2]:
import os
import time
import sys
import warnings

sys.path.append("/home/prakank/anaconda3/lib/python3.8/site-packages/")

import scipy
import numpy as np
from cvxopt import matrix, solvers

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

from python.svmutil import *

In [8]:
BINARY_CLASSIFICATION = True
LAST_DIGIT = 1
BASE_DIR = "../"
train_path = os.path.join(BASE_DIR, "data", "mnist","train.csv")
test_path  = os.path.join(BASE_DIR, "data", "mnist","test.csv")

In [9]:
def load_data(filename, Binary):
    data = np.genfromtxt(filename,delimiter=',')
    data_x = data[:,:784]/255
    data_y = data[:,784]
    data_y = data_y.reshape((data_y.shape[0],1))
    
    if Binary:
        data_x = data_x[(data_y==LAST_DIGIT).reshape(-1) | ( data_y==((LAST_DIGIT+1)%10) ).reshape(-1)]
        data_y = data_y[(data_y==LAST_DIGIT).reshape(-1) | ( data_y==((LAST_DIGIT+1)%10) ).reshape(-1)]
        data_y = -1.0*(data_y==LAST_DIGIT) + 1.0*(data_y==((LAST_DIGIT+1)%10))
        
    return data_x,data_y

In [10]:
def linear_kernel(X,y):
    mat = np.array(X*y)
    return np.dot(mat, mat.T)

def gaussian_kernel_element(X1,X2,gamma=0.05):
    return np.exp(-(np.linalg.norm(X1-X2)**2) * gamma)

def gaussian_kernel(X,gamma=0.05):
    X_sq   = np.sum(np.multiply(X, X),axis=1, keepdims=True)
    kernel_partial = X_sq + X_sq.T
    kernel_partial = kernel_partial - 2*np.dot(X,X.T)
    kernel = np.power(np.exp(-gamma),kernel_partial)
    return kernel

    kernel = np.zeros((X.shape[0],X.shape[0]))
    for i in range(X.shape[0]):
        for j in range(X.shape[0]):
            kernel[i,j] = gaussian_kernel_element(X[i],X[j],gamma)
    return Kernel

In [11]:
class color:
    PURPLE = '\033[95m'
    CYAN = '\033[96m'
    DARKCYAN = '\033[36m'
    BLUE = '\033[94m'
    GREEN = '\033[92m'
    YELLOW = '\033[93m'
    RED = '\033[91m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'
    END = '\033[0m'

In [12]:
class SVM:
    def __init__(self,kernel,C,threshold=1e-5,gamma=0.05):
        if kernel == "linear":
            self.kernel = linear_kernel
        else:
            self.kernel = gaussian_kernel
            
        self.C = float(C)
        self.threshold = threshold
        self.gamma = gamma
    
    def train(self, X_train, Y_train):
        # minimizing function
        P = 0
        if self.kernel == linear_kernel:
            P = self.kernel(X_train,Y_train)
        elif self.kernel == gaussian_kernel:
            kernel = matrix(self.kernel(X_train, self.gamma))
            P = (kernel*Y_train)*(Y_train.T)

        P = matrix(.5 * (P + P.T))  # Just to be on the safe side (ensuring P is symmetric)
        q = matrix(-1.0*np.ones((X_train.shape[0],1)))
        c = 0.0
        
        # Inequalities
        pos = np.diag(np.ones(X_train.shape[0]))
        neg = np.diag(-np.ones(X_train.shape[0]))
        G   = matrix( np.vstack((neg,pos)) )
        
        zer   = np.zeros((X_train.shape[0]))
        c_val = self.C*np.ones(X_train.shape[0])
        h     = matrix(np.concatenate((zer,c_val)))
        
        # Equality
        A = matrix(Y_train.reshape((1,Y_train.shape[0])))
        b = matrix(0.0)
        
        solvers.options['show_progress'] = False
        sol = solvers.qp(P, q, G, h, A, b);
        
        self.alpha_whole = np.array(sol['x']).copy()
        alpha = np.array(sol['x']).copy()        
        
        self.support_vector_flag = (alpha > self.threshold).reshape(-1)
        # self.support_vector_indices = (np.arange(len(alpha)))[self.support_vector_flag]
        self.alpha = alpha[self.support_vector_flag]
        self.support_vector_x = X_train[self.support_vector_flag]
        self.support_vector_y = Y_train[self.support_vector_flag]
        
        # return self.support_vector_x, self.support_vector_y, self.alpha
        
        if self.kernel == linear_kernel:
            w_partial = self.support_vector_x * self.support_vector_y
            self.w = np.sum(w_partial * self.alpha, axis=0)
            
            b1 = np.min(X_train[(Y_train == 1).reshape(-1)] * self.w)
            b2 = np.max(X_train[(Y_train ==-1).reshape(-1)] * self.w)
            self.b  = (b1+b2)*(-0.5)
        else:
            self.w = None
            
            b1 = float("inf")
            b2 = -float("inf")
            
            kernel = gaussian_kernel(self.support_vector_x,self.gamma)
            
            for i in range(len(self.alpha)):
                kernel_partial = (kernel[i,:]).reshape(-1)
                val = ((self.alpha).reshape(-1)) * ((self.support_vector_y).reshape(-1)) * kernel_partial
                val = np.sum(val)
                # for j in range(len(self.alpha)):
                #     val += self.alpha[j]*self.support_vector_y[j]*kernel[i][j]
                if self.support_vector_y[i] == 1:
                    b1 = min(b1, val)
                else:
                    b2 = max(b2,val)
            
            if b1 == float("inf"):
                b1 = 0
            if b2 == -float("inf"):
                b2 = 0                
            self.b = (b1+b2)*(-0.5)

        return self.alpha, self.w, self.b
    
    def predict2(self,X_train,Y_train,X_test,Y_test):
        prediction = np.asmatrix(np.ones((X_test.shape[0],1),dtype=int))      
        raveled = np.asmatrix(self.alpha_whole)
        
        print(raveled.shape, self.alpha_whole.shape)
        
        Xtrain = np.sum(np.multiply(X_train,X_train),axis=1)
        Xtest  = np.sum(np.multiply(X_test,X_test),axis=1)
        X_train_X_test = np.dot(X_train,X_test.T)

        alpha_x_label = np.multiply(Y_train,np.multiply(raveled,raveled>self.threshold))
        langrangian_params = np.nonzero(raveled>self.threshold)[0]

        if len(langrangian_params)==0:
            print("No support vectors found for tolerance value= " + str(self.threshold))
        else:
            b = 0
            for sv_idx in langrangian_params:
                b+=(Y_train[sv_idx,0] - np.sum(np.multiply(alpha_x_label,np.exp(-1*self.gamma*np.sum(np.multiply(X_train-X_train[sv_idx,:],X_train-X_train[sv_idx,:]),axis=1)))))
            b = b/(float(len(langrangian_params)))
            
            for i in range(X_test.shape[0]):
                prediction[i,0] = np.sign(np.sum(np.multiply(alpha_x_label,np.exp(-1*self.gamma*(Xtrain - 2*X_train_X_test[:,i] + Xtest[i,0])))) + b)

        return prediction    

    def predict(self, X_test):   
        if self.kernel == linear_kernel:
            Y_pred = (np.dot(X_test,self.w)) + self.b
        else:
            Y_pred = np.zeros((X_test.shape[0]))
            alpha  = self.alpha.reshape(-1)
            support_vector_y = self.support_vector_y.reshape(-1)

            # kernel = 

            for i in range(X_test.shape[0]):
                mat = np.array(list(map(lambda x: gaussian_kernel_element(x,X_test[i],gamma=self.gamma),self.support_vector_x)))
                Y_pred[i] = np.sum(mat*alpha*support_vector_y) + self.b                      

        Y_pred = Y_pred.reshape(-1)
        Y_pred = np.array(list(map(lambda x: -1 if x<0 else 1,Y_pred)))
        return Y_pred

In [13]:
# Part A
BINARY_CLASSIFICATION = True
if BINARY_CLASSIFICATION:
    X_train, Y_train = load_data(train_path, BINARY_CLASSIFICATION)
    X_test, Y_test  = load_data(test_path, BINARY_CLASSIFICATION)        

In [8]:
print(X_train.shape, Y_train.shape)

(4000, 784) (4000, 1)


In [14]:
# Part A -> (i)
svm_lin = SVM(kernel="linear",C=1)
alpha,w,b = svm_lin.train(X_train,Y_train)

print("Support Vector: {}\nList: {}".format(len(alpha), alpha))
print("W:",w,"\n")
print("b:",b)

Support Vector: 159
List: [[1.05617414e-01]
 [5.85350336e-02]
 [4.95495286e-02]
 [2.89173373e-03]
 [1.59593510e-01]
 [2.78152816e-01]
 [8.65459276e-02]
 [1.00000000e+00]
 [4.66424832e-01]
 [5.22162684e-01]
 [2.23429919e-01]
 [1.58398078e-01]
 [9.14809952e-03]
 [4.96733751e-02]
 [1.39922350e-01]
 [4.05487936e-03]
 [1.08793361e-01]
 [6.25883828e-03]
 [3.19792439e-02]
 [1.40868207e-01]
 [3.47977548e-01]
 [2.31937190e-02]
 [2.03690134e-01]
 [3.84446374e-01]
 [1.49192357e-01]
 [4.04955424e-01]
 [3.84107924e-02]
 [3.99877566e-01]
 [3.04201392e-02]
 [3.13275696e-01]
 [5.14247797e-02]
 [3.80939158e-02]
 [4.52554745e-01]
 [6.78845792e-01]
 [2.97845295e-01]
 [4.43402958e-01]
 [1.67277469e-01]
 [7.19555586e-02]
 [1.80891319e-01]
 [1.00485681e-02]
 [3.16665582e-01]
 [8.27972049e-02]
 [1.93059957e-01]
 [9.17706428e-02]
 [9.53784362e-02]
 [2.57158214e-01]
 [1.80716955e-01]
 [2.54709704e-02]
 [7.03333396e-02]
 [4.16004971e-02]
 [2.72076668e-03]
 [1.66459525e-01]
 [2.10333714e-01]
 [1.15593170e-01]
 [

In [15]:
w.shape

(784,)

In [17]:
Y_validation = svm_lin.predict(X_train)
print("Validation Accuracy: {}%".format(round(100*accuracy_score(Y_validation,Y_train),3)))

Y_pred = svm_lin.predict(X_test)
print("Test Set Accuracy (Linear Kernel): {}%".format(round(100*accuracy_score(Y_pred,Y_test),3)))

Validation Accuracy: 97.975%
Test Set Accuracy (Linear Kernel): 98.477%


In [18]:
# Part A -> (ii)
svm_gau = SVM(kernel="gaussian",C=1)
alpha,w,b = svm_gau.train(X_train,Y_train)

print("Support Vector: {}\nList: {}".format(len(alpha), alpha))
# print("W:",w,"\n")
print("b:",b)

Support Vector: 867
List: [[1.16956991e-02]
 [1.47274402e-04]
 [9.99999994e-01]
 [6.90274826e-02]
 [9.44218471e-02]
 [8.99039114e-01]
 [3.51481788e-03]
 [4.27276253e-02]
 [6.03387922e-01]
 [1.27433346e-02]
 [7.57699927e-02]
 [5.76902550e-02]
 [2.09097995e-02]
 [4.05430717e-02]
 [2.93609754e-01]
 [1.01651519e-02]
 [1.85209200e-02]
 [1.47250843e-01]
 [2.24557962e-01]
 [3.10705551e-01]
 [1.28595117e-02]
 [7.79233913e-03]
 [2.71556889e-01]
 [2.29185423e-01]
 [5.95212190e-02]
 [2.75903820e-01]
 [7.75604083e-01]
 [2.77860433e-02]
 [8.78559978e-02]
 [8.15051365e-02]
 [1.27390534e-01]
 [5.44559446e-03]
 [9.99999999e-01]
 [1.01297284e-01]
 [9.99999997e-01]
 [1.19240472e-01]
 [8.10804399e-03]
 [3.14259397e-01]
 [1.37131099e-03]
 [5.42587037e-01]
 [1.80498711e-02]
 [1.71792401e-02]
 [2.31852914e-02]
 [1.02280972e-01]
 [4.72171923e-02]
 [1.17609951e-02]
 [1.59316126e-02]
 [3.59296191e-01]
 [9.78659271e-02]
 [7.88131455e-03]
 [4.88339935e-02]
 [4.34351940e-02]
 [1.26871181e-01]
 [9.99999993e-01]
 [

In [19]:
Y_validation = svm_gau.predict(X_train)
print("Validation Accuracy: {}%".format(round(100*accuracy_score(Y_validation,Y_train),3)))

Y_pred = svm_gau.predict(X_test)
print("Test Set Accuracy (Gaussian Kernel): {}%".format(round(100*accuracy_score(Y_pred,Y_test),3)))

Validation Accuracy: 100.0%
Test Set Accuracy (Gaussian Kernel): 99.723%


In [22]:
# Part a -> (iii)
model1 = svm_train(Y_train.reshape(-1), X_train, '-s 0 -c 1 -t 0')
label_predict, accuracy,  decision_values  =svm_predict(Y_test.reshape(-1),X_test,model1, '-q');
label_predict2, accuracy2, decision_values2=svm_predict(Y_train.reshape(-1),X_train,model1, '-q');
print("{}Linear Score: {}% {}".format(color.BOLD,round(100*accuracy_score(label_predict,Y_test),3),color.END ) )
print("{}Linear Score: {}% {}".format(color.BOLD,round(100*accuracy_score(label_predict2,Y_train),3),color.END ) )

..*..*
optimization finished, #iter = 4258
nu = 0.008999
obj = -18.274204, rho = 1.219513
nSV = 158, nBSV = 2
Total nSV = 158
Linear Score: 99.031% 
Linear Score: 100.0% 


In [26]:
print(len(decision_values))

4000


In [28]:
model2 = svm_train(Y_train.reshape(-1), X_train, '-s 0 -c 1 -t 2 -g 0.05')
label_predict_g, accuracy_g, decision_values_g   =svm_predict(Y_test.reshape(-1),X_test,model2,'-q');
label_predict_g2, accuracy_g2, decision_values_g2=svm_predict(Y_train.reshape(-1),X_train,model2, '-q');
print("{}RBF Score: {}% {}".format(color.BOLD,round(100*accuracy_score(label_predict_g,Y_test),3),color.END ) )
print("{}RBF Score: {}% {}".format(color.BOLD,round(100*accuracy_score(label_predict_g2,Y_train),3),color.END ) )

*.*
optimization finished, #iter = 1288
nu = 0.040537
obj = -87.883228, rho = -0.890975
nSV = 848, nBSV = 40
Total nSV = 848
RBF Score: 99.585% 
RBF Score: 99.975% 


In [9]:
# Part B
BINARY_CLASSIFICATION = False

if not BINARY_CLASSIFICATION:
    X_train, Y_train = load_data(train_path, BINARY_CLASSIFICATION)
    X_test, Y_test  = load_data(test_path, BINARY_CLASSIFICATION)

In [10]:
def svm_load_data(X,Y,d1,d2):
    X_data = X[(Y == d1).reshape(-1) | (Y == d2).reshape(-1)]
    Y_data = Y[(Y == d1).reshape(-1) | (Y == d2).reshape(-1)]
    Y_data = -1.0*(Y_data == d1) + 1.0*(Y_data == d2)
    return X_data, Y_data

In [12]:
# Part B -> (i)

def one_vs_one_classifier_train(max_val_, X_train, Y_train, X_test, Y_test):
    svm_models = {}
    max_val = max_val_
    for i in range(max_val):
        for j in range(i+1,max(2,max_val)):
            start = time.time()
            print("\nTraining Phase: {" + str(i) + ", " + str(j) + "}")
            
            separator = "_"
            index = str(i) + separator + str(j)
            svm_models[index] = {}
            svm_models[index]["X_train"], svm_models[index]["Y_train"] = svm_load_data(X_train,Y_train,i,j)
            svm_models[index]["svm_gaussian"] = SVM(kernel="gaussian",C=1)
            svm_models[index]["params"] = svm_models[index]["svm_gaussian"].train(svm_models[index]["X_train"], svm_models[index]["Y_train"])

            end = time.time()
            print("Time: {}s".format(round(end-start,5)))

            svm_models[index]["time"] = round(end-start,5)
            
    return svm_models

def one_vs_one_classifier_pred(max_val, pred_type, svm_models, X, Y):
    y_pred_temp = np.zeros((X.shape[0], 10))
    print()
    for i in range(max_val):
        for j in range(i+1,max(2,max_val)):
            start = time.time()
            print("Prediction Phase (" + pred_type + "): {" + str(i) + ", " + str(j) + "}",end="  ")
            
            separator = "_"
            index = str(i) + separator + str(j)
            pred_string = pred_type + "_predicted"
            
            svm_models[index][pred_string] = y_pred = svm_models[index]["svm_gaussian"].predict(X)
            for k in range(y_pred.shape[0]):
                if y_pred[k] == -1:
                    y_pred_temp[k][j] += 1
                else:
                    y_pred_temp[k][i] += 1
            
            end = time.time()
            print("Time: {}s".format(round(end-start,5)))
            
            svm_models[index]["time"] += round(end-start,5)
            
    y_pred_final = [0]*y_pred_temp.shape[0]
    
    for i in range(len(y_pred_final)):
        y_pred_final[i] = 9-np.argmax((y_pred_temp[i,:])[::-1])
    
    svm_models[pred_type + "_predicted"] = np.array(y_pred_final)
    return svm_models
    
def one_vs_one_classifier(max_val_, X_train, Y_train, X_test, Y_test):
    max_val = max_val_
    
    start = time.time()
    
    svm_models = one_vs_one_classifier_train(max_val, X_train, Y_train, X_test, Y_test)
    return svm_models

    svm_models = one_vs_one_classifier_pred (max_val, "val", svm_models, X_test, Y_test)
    
    end = time.time()
    svm_models["time"] = round(end-start,5)
    
    print("Test Accuracy (Gaussian Kernel): {}%".format(round(100*accuracy_score(svm_models["val_predicted"], Y_test), 5)))
    
    return svm_models

In [13]:
max_val = 3
svm_models = one_vs_one_classifier(max_val, X_train, Y_train, X_test, Y_test)


Training Phase: {0, 1}
Time: 45.92458s

Training Phase: {0, 2}
Time: 30.19063s

Training Phase: {1, 2}
Time: 43.6299s


In [27]:
print(len(svm_models["0_1"]["params"][0]))
print(len(svm_models["0_2"]["params"][0]))
print(len(svm_models["1_2"]["params"][0]))
x_temp,y_temp = svm_load_data(X_test, Y_test,1,2)
print(x_temp.shape,y_temp.shape)
# y_p = svm_models["1_2"]["svm_gaussian"].predict(X_test)
# print((svm_models["1_2"]["params"][0]))

775
1703
867
(2167, 784) (2167, 1)


In [26]:
count = 0
for i in range(y_p.shape[0]):
    if (y_p[i] == Y_test[i]):
        count+=1
print(count)

5


In [43]:
print(svm_models["0_1"]["val_predicted"])
print(svm_models["0_2"]["val_predicted"])
print(svm_models["1_2"]["val_predicted"])
x = svm_models["1_2"]["val_predicted"]
y = list(Y_test.astype('int'))
print(len(x), len(y))
count=0
for i in range(len(y)):
    if x[i] == y[i]:
        count+=1
print(count)

[-1 -1  1 ... -1 -1 -1]
[ 1  1  1 ...  1 -1 -1]
[ 1  1 -1 ...  1  1  1]
10000 10000
5


In [ ]:
    

    # y_pred_train = np.zeros((X_train.shape[0], 10, 10))
    # print()
    # for i in range(max_val):
    #     break
    #     for j in range(i+1,max(2,max_val)):
    #         start = time.time()
    #         print("Prediction Phase (Validation): {" + str(i) + ", " + str(j) + "}",end="  ")

    #         separator = "_"
    #         index = str(i) + separator + str(j)
    #         # y_pred = svm_models[index]["svm_gaussian"].predict2(svm_models[index]["X_train_" + index],svm_models[index]["Y_train_" + index],X_train,Y_train)
    #         y_pred = svm_models[index]["svm_gaussian"].predict(X_train)
    #         y_pred_train[:,i,j] = (y_pred-1)/(-2)
    #         y_pred_train[:,j,i] = (y_pred+1)/(2)
            
    #         end = time.time()
    #         print("Time: {}s".format(round(end-start,3)))
    
    # y_temp1 = y_pred_train.copy()
    # y_pred_train = np.abs(y_pred_train)
    # y_pred_train = np.argmin(np.sum(y_pred_train, axis=1), axis = 1)
        
    # y_pred_train = np.sum(y_pred_train,axis=2)
    # y_pred_train = y_pred_train[:,::-1]
    # y_pred_train = np.argmax(y_pred_train,axis=1)
    
    # print("Validation Accuracy (Gaussian Kernel): {}%".format(round(100*accuracy_score(y_pred_train, Y_train), 3)))
    
    # y_pred_test_temp = np.zeros((X_test.shape[0], 10))
    # print()
    # for i in range(max_val):
    #     for j in range(i+1,max(2,max_val)):
    #         start = time.time()
    #         print("Prediction Phase (Test set): {" + str(i) + ", " + str(j) + "}",end="  ")

    #         # y_pred = svm_models[index]["svm_gaussian"].predict2(X_train,Y_train,X_test,Y_test)
    #         y_pred = svm_models[index]["svm_gaussian"].predict(X_test)
    #         for k in range(y_pred.shape[0]):
    #             if y_pred[k] == -1:
    #                 y_pred_test_temp[k][j] += 1
    #             else:
    #                 y_pred_test_temp[k][i] += 1
    #         # y_pred_test[:,i,j] = (y_pred-1)/(-2)
    #         # y_pred_test[:,j,i] = (y_pred+1)/(2)
            
    #         end = time.time()
    #         print("Time: {}s".format(round(end-start,3)))
            
    # y_pred_test = [0]*y_pred_test_temp.shape[0]
    # for i in range(len(y_pred_test)):
    #     y_pred_test[i] = 9-np.argmax((y_pred_test_temp[i,:])[::-1])
    
    
    # y_temp2 = y_pred_test.copy()
    # y_pred_test = np.sum(y_pred_test,axis=2)
    # y_pred_test = y_pred_test[:,::-1]
    # y_pred_test = np.argmax(y_pred_test,axis=1)
    
    # print("Validation Accuracy (Gaussian Kernel): {}%".format(round(100*accuracy_score(y_pred_train, Y_train), 3)))
    # print("Test Accuracy (Gaussian Kernel): {}%".format(round(100*accuracy_score(y_pred_test, Y_test), 3)))
    # return svm_models, np.array(y_pred_test)

    # return y_temp1, y_temp2

In [13]:
start = time.time()
y_pred_train, y_pred_test = one_vs_one_classifier(X_train, Y_train, X_test, Y_test)
end = time.time()
print("\n\nGlobal Time: {}".format(round(end-start,7)))


Training Phase: {0, 1}
Time: 57.542s

Training Phase: {0, 2}
Time: 41.705s

Training Phase: {0, 3}
Time: 40.799s

Training Phase: {0, 4}
Time: 44.142s

Training Phase: {0, 5}
Time: 39.039s

Training Phase: {0, 6}
Time: 45.265s

Training Phase: {0, 7}
Time: 44.157s

Training Phase: {0, 8}
Time: 41.189s

Training Phase: {0, 9}
Time: 41.249s

Training Phase: {1, 2}
Time: 37.67s

Training Phase: {1, 3}
Time: 37.316s

Training Phase: {1, 4}
Time: 35.041s

Training Phase: {1, 5}
Time: 37.675s

Training Phase: {1, 6}
Time: 34.198s

Training Phase: {1, 7}
Time: 37.438s

Training Phase: {1, 8}
Time: 34.582s

Training Phase: {1, 9}
Time: 37.626s

Training Phase: {2, 3}
Time: 28.896s

Training Phase: {2, 4}
Time: 29.403s

Training Phase: {2, 5}
Time: 29.879s

Training Phase: {2, 6}
Time: 29.761s

Training Phase: {2, 7}
Time: 31.537s

Training Phase: {2, 8}
Time: 28.202s

Training Phase: {2, 9}
Time: 30.94s

Training Phase: {3, 4}
Time: 29.37s

Training Phase: {3, 5}
Time: 29.452s

Training Phase

In [21]:
# for i in range(10):
#     print(y_pred_test[i,:,:])

y_temp = np.sum(y_pred_test,axis=2)
y_temp = y_temp[:,::-1]
y_temp = np.argmax(y_temp,axis=1)
L = [0 for i in range(10)]
for i in y_temp:
    L[int(i)]+=1
print(L)

[3381, 0, 0, 0, 0, 0, 0, 0, 0, 6619]


In [14]:
print(y_pred_train, y_pred_test)

[[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 ...

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]] [[[ 0. -0.

In [15]:
confusion_matrix1 = np.zeros((10,10))
for i in range(Y_train.shape[0]):
    confusion_matrix1[Y_train[i]][y_pred_train[i]] += 1
print("Training set")
print(confusion_matrix1.astype(int))

confusion_matrix1 = np.zeros((10,10))
for i in range(Y_test.shape[0]):
    confusion_matrix1[Y_test[i]][y_pred_test[i]] += 1
print("Test set")
print(confusion_matrix1.astype(int))

# label_predict <- Part B's b (LIBSVM)

# confusion_matrix2 = np.zeros((10,10))
# for i in range(Y_test.shape[0]):
#     confusion_matrix2[Y_test[i]][label_predict[i]] += 1
# print(confusion_matrix2.astype(int))


IndexError: arrays used as indices must be of integer (or boolean) type

In [ ]:
# Part B -> (ii)
model = svm_train(Y_train.reshape(-1), X_train, '-s 0 -c 1 -t 2 -g 0.05 -q')
label_predict1, accuracy, decision_values=svm_predict(Y_train.reshape(-1),X_train,model,'-q')
print("Train Accuracy :", accuracy_score(label_predict1, Y_train))

label_predict2, accuracy, decision_values=svm_predict(Y_test.reshape(-1),X_test,model, '-q')
print("Test Accuracy :", accuracy_score(label_predict2, Y_test))

Train Accuracy : 0.9992
Test Accuracy : 0.9723
